In [10]:
# Arkansas
import csv
#import pandas as pd
#import numpy as np
import json
file = 'ftp://www.aogc2.state.ar.us/AOGC.zip'
# downlaoded MS Access DB from the above, exported to CSV
# Then exported the following tables to CSV with headers
tables = ['history.txt', 'WellMaster_SideTrack_Location.txt', 'WellMaster.txt']
res = 'data/ar.bin'
types = {'Natural Gas - Dry', 'Coal Bed Methane', 'Enhanced Oil Recovery', 'Oil - Production', 'Oil-Gas Combination Well'}
statuses = {'Dry And Abandoned', 'Plugged and Abandoned', 'Domestic Well - Gas', 'Active', 'Producing', 'Temporarily Abandoned', 'Spudded', 'Abandoned/Orphaned Well', 'Completed'}


with open('capture/ar/WellMaster.csv', 'rb') as f:
    reader = csv.DictReader(f)
    master = [row for row in reader]
#with open('capture/ar/WellMaster_SideTrack.txt', 'rb') as f:
#    reader = csv.DictReader(f)
#    sidetrack = [row for row in reader]
#with open('capture/ar/WellMaster_SideTrack_Location.txt', 'rb') as f:
#    reader = csv.DictReader(f)
#    locations = [row for row in reader]
with open('capture/ar/history.csv', 'rb') as f:
    reader = csv.DictReader(f)
    history = [row for row in reader]
        
with open(r'capture\ar\data.json', 'r') as f:
    rows = json.load(f)

rows = [row for row in rows if row['Well Type'] in types and row['Well Status'] in statuses]

for row in rows:
    api = row['API Well No']
    row['API_WellNo'] = ''.join(e for e in api if e.isalnum()) # strip dashes
    
print master[0]
print history[0]
print 'records:', str(len(rows)), '. first row:', rows[0]




{'RangeDir': 'W', 'TMD': '', 'Section': '27', 'Wl_Status': 'DA', 'DTD': '2076', 'PB_MD': '', 'Wl_Permit': '00001', 'Latitude': '', 'DepthToBeDrld': '', 'Dt_Status': '3/21/1924 0:00:00', 'Elevation': '', 'Field_Name': 'WILDCAT', 'FtEW': '', 'Pool_Name': 'WILDCAT', 'FtEWDir': '', 'Ref_Const': '', 'Longitude': '', 'TwpDir': 'S', 'CoName': 'Peer oil Corp', 'Well_Typ': 'OIL', 'RangeNum': '16.00', 'FtNSDir': '', 'QQ': '', 'Q': '', 'FtNS': '', 'API_WellNo': '03103805360000', 'TwpNum': '15.00', 'County': 'OUACHITA', 'PRUID': '', 'QQQ': '', 'Well_No': '1', 'Well_Nm': 'Mullens 1'}
{'Dt_RptReqd': '', 'rowguid': '', 'Dt_Effect': '7/1/1999 0:00:00', 'Dt_RptRcvd': '', 'Dt_Mod': '12/6/1999 7:54:00', 'API_WellNo': '03001000030000', 'UserId': '', 'Sbsqnt_Rpt': '', 'Typ_Work': 'CHOP', 'Cmmnt': 'Operator changed from 2668, SEAGULL ENERGY E&P INC. to 4032, Arkoma Holding Corporation', 'Typ_Form': ''}
records: 42783 . first row: {u'API Well No': u'03-131-10854-00-00', u'Section': u'24', u'Ft EW': u'565', u

In [11]:
from datetime import datetime
for row in rows:
    dates = [i['Dt_Status'] for i in master if i['API_WellNo'] == row['API_WellNo']]
    dates += [i['Dt_Effect'] for i in history if i['API_WellNo'] == row['API_WellNo']]
    dates = sorted([datetime.strptime(d, '%m/%d/%Y %H:%M:%S') for d in dates if d != ''])
    #for date in dates:
    #    print str(date),
    #print '\n'
    if len(dates) > 0 and dates[0]:
        row['min_date'] = dates[0]
    

In [16]:
import math, array
data = []
def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]
for row in rows:
    if row['Longitude'] == '' or row['Latitude'] == '':
        continue
    if 'min_date' not in row:
        continue
    try:
        lon = float(row['Longitude'])
        lat = float(row['Latitude'])
        x,y = LonLatToPixelXY([lon,lat])
    except ValueError:
        print 'error converting coordinate for: ', row
        continue

    epochtime = (row['min_date'] - datetime(1970, 1, 1)).total_seconds()
    data += [x,y,epochtime]

#print data
print 'processed ', str(len(data)/3), ' wells'
f.close()
array.array('f', data).tofile(open(res, 'wb'))
    

processed  28892  wells


In [21]:
# Arkansas
# In reply to your question we have an FTP site where we maintain a complete list of wells. It is in Access Database format but could be imported to Excel. We also have Shape file stored there as well. The file you are looking for is “AOGC.zip”, or if you want the shape files go to the folder GIS_Files inside the FTP site.
# FTP site: ftp://www.aogc2.state.ar.us/
# download AOGC.zip

# download XLS files by iterating through well types
# at: http://www.aogc2.state.ar.us/welldata/Wells/Default.aspx
# Oil and Gas wells required up to 12 hours to download
# Files are actually HTML. Converted to CSV using Excel

from datetime import datetime
import csv, array

# Takes an interative approach.

import glob, random, json, os.path
import requests
from lxml import html
files = glob.glob(r'capture\ar\*.csv')
tmp_file = r'capture\ar\data.json'
res = 'data/ar-oil.bin'
types = ['Natural Gas - Dry', 'Coal Bed Methane', 'Enhanced Oil Recovery', 'Oil - Production', 'Oil-Gas Combination Well']
statuses = {'Dry And Abandoned', 'Plugged and Abandoned', 'Domestic Well - Gas', 'Active', 'Producing', 'Temporarily Abandoned', 'Spudded', 'Abandoned/Orphaned Well', 'Completed'}

print files
# then use API number to scrape dates from:
# http://www.aogc2.state.ar.us/AOGConline/ED.aspx?KeyName=API_WELLNO&KeyValue=03145101670000&KeyType=STRING&DetailXML=WellDetails.xml

print_count = 0
rows = []
apis = set()

try:
    with open(tmp_file) as f:
        rows = json.load(f)
    apis = set([i['API Well No'] for i in rows])
    print len(apis), 'apis in set'
except IOError:
    print 'no json file to load from'

for file in files:
    with open(file, 'rb') as f:
        reader = csv.DictReader(f)
        #for row in reader:
        #    if row['API Well No'] not in apis:
        #        rows.append(row)
        #        print_count += 1
        rows += [row for row in reader if row['API Well No'] not in apis]
        # look, ma! I'm learning list comprehensions!
if len(rows) > 0:
    print rows[random.randint(0,len(rows))]

print 'writing', str(len(rows)), 'rows to', tmp_file
    # capture our progress so far
with open(tmp_file, 'w') as f:
    json.dump(rows, f)



['capture\\ar\\ar-cbm-WellData.csv', 'capture\\ar\\ar-enh-oil-recv-WellData.csv', 'capture\\ar\\ar-natural-gas-WellData.csv', 'capture\\ar\\ar-oil-production-WellData.csv']
no json file to load from
{'API Well No': '03-071-10755-00-00', 'Section': '13', 'Ft EW': '', 'Field': 'SPADRA', 'Well Status': 'Plugged and Abandoned', 'Latitude': '35.43715', 'Twp Num': '9', 'Rng Num': '23', 'PB TD': '6875', 'Well Type': 'Natural Gas - Dry', 'TD': '7219', 'NS': '', 'Zone/Pool': 'RALPH BARTON', 'Longitude': '-93.4007', 'Well Name': 'Willis 1-13', 'Rng Dir': 'W', 'Permit': '36288', 'EW': '', 'Status Date': '8/16/2010', 'Twp Dir': 'N', 'County': 'JOHNSON', 'Operator': 'Shields Operating, Inc', 'Elev Gr': '458', 'Ft NS': '', 'Location': '9N-23W'}
writing 44081 rows to capture\ar\data.json


In [25]:
tmp_file = r'capture\ar\data.json'
rows = []
try:
    with open(tmp_file, 'rb') as f:
        rows = json.load(f)
except IOError:
    print 'no json file to load from'

remaining = len([_ for row in rows if 'date' not in row])

count, success = 0, 0
print 'scraping dates for ', remaining, ' records:',
for row in rows:
    if row['Latitude'] == '' or row['Longitude'] == '':
        continue
    if row['Well Status'] not in statuses:
        continue
    if 'date' in row:
        continue # already downloaded the date!
    count += 1
    api = row['API Well No']
    api = ''.join(e for e in api if e.isalnum()) # strip dashes
    url = 'http://www.aogc2.state.ar.us/AOGConline/ED.aspx?KeyName=API_WELLNO&KeyValue=%s&KeyType=STRING&DetailXML=WellDetails.xml' % api
    try: 
        page = requests.get(url)
    except ConnectionError as e:
        print 'Error loading:', e
        continue
        
    if page.content[45:73] == r'<title>Runtime Error</title>':
        print 'Error loading:', url
        break
    tree = html.fromstring(page.content)
    spud_date = tree.xpath('//*[@id="EDI0"]/table/tr[2]/td[4]/text()')
    date = None
    if spud_date:
        date = spud_date[0]
    elif row['Status Date'] != '':
        date = row['Status Date']
    else:
        permit_date = tree.xpath('//*[@id="EDI0"]/table/tr[2]/td[2]/text()')
        if permit_date:
            date = permit_date[0]
        else:
            status_date = tree.xpath('//*[@id="ED"]/table[1]/tr[2]/td[10]/text()')
            if status_date:
                date = status_date[0]
    if date and date != '' and date != ' ':
        date = datetime.strptime(date, '%m/%d/%Y')
        success += 1
    row['date'] = str(date)
    #if count % (len(rows) // 100) == 0:
        #print '*',           
    #    print str(count // len(rows)),
    print '*',
    if count % (len(rows) // 100) == 0:
        print '[' + str(count // len(rows)) + ']',
        
print '\nRetrieved dates for', str(success), 'of', str(count), 'records.'

# capture our progress so far
with open(tmp_file, 'wb') as f:
    json.dump(rows, f)
print 'finished'

scraping dates for  43987  records: Error loading: http://www.aogc2.state.ar.us/AOGConline/ED.aspx?KeyName=API_WELLNO&KeyValue=03131114120000&KeyType=STRING&DetailXML=WellDetails.xml

Retrieved dates for 0 of 97 records.
finished


In [ ]:
def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

data = []   
for row in rows:
    lat = row['Latitude']
    lon = row['Longitude']
    x,y = LonLatToPixelXY([lon,lat])
    epochtime = (row['date'] - datetime(1970, 1, 1)).total_seconds()
    data += [x,y,epochtime] 


print 'processed ', str(len(data)/3), ' wells'
f.close()
array.array('f', data).tofile(open(res, 'wb'))